In [1]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")
import json
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer

In [2]:
import operator
import pandas as pd
fileList=['SentenceEmbeddingsStsbRobertaLarge','TextualEntailment','DistilBert','GPT','GPTLarge']
#fileList=['SentenceEmbeddingsStsbRobertaLarge','DistilBert','GPT']
resultDf=pd.DataFrame()
for num,fileName in enumerate(fileList):
    print(fileName)
    dfs=pd.read_excel("FinalDatasetAuto"+fileName+".xlsx",index_col=0)
    dfs['arg0']=dfs['arg0'].replace(np.NaN,'')
    dfs['arg1']=dfs['arg1'].replace(np.NaN,'')
    dfs['PP']=dfs['PP'].replace(np.NaN,'')
    columnList=['arg0','V','arg1','PP']
    for index,i in enumerate(dfs['adj/noun']):
        result=''
        for column in columnList:
            if dfs.loc[index,column]!='':
                    result=result+dfs.loc[index,column]+" "
        dfs.loc[index,'gold']=result
    for index,i in enumerate(dfs['adj/noun']):
        if index+1<len(dfs['adj/noun']) and dfs.loc[index,'adj/noun']==dfs.loc[index+1,'adj/noun'] and dfs.loc[index,'n_v']==dfs.loc[index+1,'n_v'] and dfs.loc[index,'prep']==dfs.loc[index+1,'prep'] and dfs.loc[index,'pobj']==dfs.loc[index+1,'pobj']:
            dfs.loc[index,'gold2']=dfs.loc[index+1,'gold']
        else:
            dfs.loc[index,'gold2']=dfs.loc[index,'gold']
    for index,i in enumerate(dfs['adj/noun']):
        #print(index)
        dfs.loc[index,'OriginalHighestScorePattern']=dfs.loc[index,'HighestScorePattern']
        if dfs.loc[index,'HighestScorePattern']=='MVpOPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='OVM'
            else:  
                dfs.loc[index,'HighestScorePattern']='MVpOPassiveVMpO'
        if dfs.loc[index,'HighestScorePattern']=='MpOVPassive':
                dfs.loc[index,'HighestScorePattern']='MpOVPassiveVMpO'
        if dfs.loc[index,'HighestScorePattern']=='OVpMPassive':
            pattern=dfs.loc[index,'HighestScorePattern']
            if "by" in dfs.loc[index,pattern].split():
                dfs.loc[index,'HighestScorePattern']='MVO'
            else:
                dfs.loc[index,'HighestScorePattern']='OVpMPassiveVOpM'
    for index,i in enumerate(dfs['gold']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                        reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold']=reducedGold.lower()
            reducedGold3=''
            for para in dfs.loc[index,'gold'].split():
                doc=nlp(para)
                for word in doc:
                    if word.tag_!='DT':
                        if word.text!='a' and word.text!='an' and word.text!='the':
                            answer=lemmatizer(word.text, NOUN)
                            reducedGold3=reducedGold3+answer[0]+" "
            dfs.loc[index,'reducedGold3']=reducedGold3.lower()
    for index,i in enumerate(dfs['gold2']):
        if not pd.isnull(i):
            doc=nlp(i)
            reducedGold=''
            for word in doc:
                if word.tag_!='DT':
                        reducedGold=reducedGold+word.lemma_+" "
            dfs.loc[index,'reducedGold2']=reducedGold.lower()
            reducedGold4=''
            for para in dfs.loc[index,'gold2'].split():
                doc=nlp(para)
                for word in doc:
                    if word.tag_!='DT':
                        if word.text!='a' and word.text!='an' and word.text!='the':
                            answer=lemmatizer(word.text, NOUN)
                            reducedGold4=reducedGold4+answer[0]+" "
            dfs.loc[index,'reducedGold4']=reducedGold4.lower()
    for index,i in enumerate(dfs['adj/noun']):
        if not pd.isnull(dfs.loc[index,'v']):
            pattern=dfs.loc[index,'HighestScorePattern']
            dfs.loc[index,'HighestScorePatternSentence']=dfs.loc[index,pattern]
            doc=nlp(dfs.loc[index,pattern])
            reducedParaphrase=''
            for word in doc:
                if word.tag_!='DT':
                        reducedParaphrase=reducedParaphrase+word.lemma_+" "
            dfs.loc[index,'reducedParaphrase']=reducedParaphrase.lower()
            reducedParaphrase2=''
            for para in dfs.loc[index,pattern].split():
                doc=nlp(para)
                for word in doc:
                    if word.tag_!='DT':
                        if word.text!='a' and word.text!='an' and word.text!='the':
                            answer=lemmatizer(word.text, NOUN)
                            reducedParaphrase2=reducedParaphrase2+answer[0]+" "
            dfs.loc[index,'reducedParaphrase2']=reducedParaphrase2.lower()
    for index,i in enumerate(dfs['adj/noun']):
        if not pd.isnull(dfs.loc[index,'v']):
            if (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold2']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold3']) or (dfs.loc[index,'reducedParaphrase']==dfs.loc[index,'reducedGold4']):
                dfs.loc[index,'FinalLabel']='True'
            elif (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold2']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold3']) or (dfs.loc[index,'reducedParaphrase2']==dfs.loc[index,'reducedGold4']):
                dfs.loc[index,'FinalLabel']='True'
            else:
                dfs.loc[index,'FinalLabel']='False'
        else:
            dfs.loc[index,'FinalLabel']='False'
    dfs=dfs.fillna('')
    for index,i in enumerate(dfs['adj/noun']):
        for columns in dfs.columns:
            dfs.loc[index,columns]=str(dfs.loc[index,columns])
    dfs=dfs.groupby(['adj/noun','n_v','prep','pobj'],sort=False)[dfs.columns[4:]].agg('//'.join).reset_index()
    dfs['final']=dfs['FinalLabel']
    for index,i in enumerate(dfs['adj/noun']):
        if 'True' in dfs.loc[index,'FinalLabel'].split("//"):
            dfs.loc[index,'FinalLabel']='True'
        else:
            dfs.loc[index,'FinalLabel']='False'
    print(dfs['FinalLabel'].value_counts())
    print("Accuracy= "+str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel'])))
    resultDf.loc[num,'model']=fileName
    resultDf.loc[num,'True']=dfs['FinalLabel'].value_counts()['True']
    resultDf.loc[num,'False']=dfs['FinalLabel'].value_counts()['False']
    resultDf.loc[num,'Accuracy']=str(dfs['FinalLabel'].value_counts()['True']/len(dfs['FinalLabel']))
    dfs.to_excel("FinalDatasetAuto"+fileName+"IgnoreMRemoveArticleMetric.xlsx")
    dfs2=dfs[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]
    with open('FinalDatasetAuto'+fileName+'IgnoreMRemoveArticleMetric.txt', 'w') as file:
    # as requested in comment
        for index,i in zip(dfs2.index,dfs2['sentence']):
            elementDict=dfs2.loc[index].to_dict()
            file.write(str(index)+". ")
            for i in elementDict:
                 if elementDict[i]!='':
                     #print (i,elementDict[i])
                     file.write(i+" : "+str(elementDict[i]))
                     file.write('\n')
            file.write('\n')
    dfs3=dfs[dfs['FinalLabel']=='False']
    dfs3=dfs3[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]
    with open('FinalDatasetAuto'+fileName+'IgnoreMRemoveArticleMetricFalse.txt', 'w') as file:
    # as requested in comment
        for index,i in zip(dfs3.index,dfs3['sentence']):
            elementDict=dfs3.loc[index].to_dict()
            file.write(str(index)+". ")
            for i in elementDict:
                 if elementDict[i]!='':
                     #print (i,elementDict[i])
                     file.write(i+" : "+str(elementDict[i]))
                     file.write('\n')
            file.write('\n')
    dfs4=dfs[dfs['FinalLabel']=='True']
    dfs4=dfs4[['sentence','adj/noun','n_v','prep','pobj','originalPattern','MVO','MVOScore','MVpO','MVpOScore','OVM','OVMScore','OVpM','OVpMScore','VOpM','VOpMScore','VpOpM','VpOpMScore','VMpO','VMpOScore','VpMpO','VpMpOScore','gold','reducedGold','HighestScorePattern','HighestScorePatternSentence','reducedParaphrase','adj/noun-label','FinalLabel']]
    with open('FinalDatasetAuto'+fileName+'IgnoreMRemoveArticleMetricTrue.txt', 'w') as file:
    # as requested in comment
        for index,i in zip(dfs4.index,dfs4['sentence']):
            elementDict=dfs4.loc[index].to_dict()
            file.write(str(index)+". ")
            for i in elementDict:
                 if elementDict[i]!='':
                     #print (i,elementDict[i])
                     file.write(i+" : "+str(elementDict[i]))
                     file.write('\n')
            file.write('\n')

SentenceEmbeddingsStsbRobertaLarge
False    336
True      33
Name: FinalLabel, dtype: int64
Accuracy= 0.08943089430894309
TextualEntailment
False    267
True     102
Name: FinalLabel, dtype: int64
Accuracy= 0.2764227642276423
DistilBert
False    249
True     120
Name: FinalLabel, dtype: int64
Accuracy= 0.3252032520325203
GPT
False    257
True     112
Name: FinalLabel, dtype: int64
Accuracy= 0.3035230352303523
GPTLarge
False    253
True     116
Name: FinalLabel, dtype: int64
Accuracy= 0.3143631436314363
